In [68]:
# %load_ext autoreload
# %autoreload 2

# import sys
# import os

# try: # When on google Colab, let's clone the notebook so we download the cache.
#     import google.colab
#     repo_path = 'dspy'
#     !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
# except:
#     repo_path = '.'

# if repo_path not in sys.path:
#     sys.path.append(repo_path)

# # Set up the cache for this notebook
# os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

# import pkg_resources # Install the package if it's not installed
# if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
#     !pip install -U pip
#     !pip install dspy-ai
#     !pip install openai~=0.28.1
#     # !pip install -e $repo_path

In [70]:
import dspy
import pydantic
import numpy as np

In [4]:
import pprint
pprint = pprint.PrettyPrinter(indent=4).pprint

In [5]:
gpt4_turbo = dspy.OpenAI(model='gpt-4-1106-preview', max_tokens=300)
gpt3_turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=300, temperature=1)

dspy.settings.configure(lm=gpt3_turbo, max_tokens=1024)

# AssessSubjectModule

This module takes a description as input and determines whether it pertains to a technical smart contract description or other topics outside of its scope.

In [73]:
import json

with open('..\examples\off_topic_descriptions\data.json', 'r') as f1,\
     open('..\examples\smart_contract_descriptions\data.json', 'r') as f2:
    d1 = json.load(f1)
    d2 = json.load(f2)

OT_descriptions = [(description, False) for description in d1['descriptions']]
SC_descriptions = []
for key in d2:
    SC_descriptions.append((d2[key]['descriptions']['description_1'], True))
    # SC_descriptions.append((d2[key]['descriptions']['description_2'], True))
    # SC_descriptions.append((d2[key]['descriptions']['description_3'], True))
    # SC_descriptions.append((d2[key]['descriptions']['description_4'], True))
combined_descriptions = OT_descriptions + SC_descriptions

# Arguments must have the same name as Signature input/output
examples = [
    dspy.Example(smart_contract_description=desc, boolean_assessment=is_valid).with_inputs("smart_contract_description")
    for desc, is_valid in combined_descriptions
]

def split_for_train_test(values, test_size=2/3.0):
    np.random.shuffle(values)
    split = int(len(values) * (1 - test_size))
    return values[:split], values[split:]

examples_for_training, examples_for_testing = split_for_train_test(examples)

# print("List of 'examples' for training: ")
# pprint(examples_for_training)
# print("\nList of 'examples' for testing: ")
# pprint(examples_for_testing)

In [77]:
class AssessSubject(dspy.Signature):
    """Verify that the text consist of a precise functional description of how a specific smart contract should work."""
    smart_contract_description: str = dspy.InputField(desc="A description of a Smart Contract")
    boolean_assessment: bool = dspy.OutputField(desc="True/False indicating if text is about Smart Contracts")

class AssessSubjectModule(dspy.Module):
    """A module to verify if the description consists of a precise functional description of how a specific smart contract should work."""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedChainOfThought(AssessSubject)

    def forward(self, smart_contract_description: str) -> bool:
        # It cannot be the boolean_assessment output parameter directly, or the teleprompting won't work.
        return self.generate_answer(smart_contract_description=smart_contract_description)

def metric(example, prediction, trace=None):

    printing = False

    if printing:
        print(f"Defined function 'metric' called on\n{' '*4}{example}")
        print(f"Generated prediction is\n{' '*4}{prediction.reasoning}")
        print(f"{' '*4}>> Real-value/Predicted-value : {example.boolean_assessment} | {prediction.boolean_assessment}\n")
        
    return example.boolean_assessment == prediction.boolean_assessment # TODO: Moved from the forward prediction return

from dspy.teleprompt import BootstrapFewShot
config = dict(max_bootstrapped_demos=len(examples_for_training)) # max_labeled_demos=4
teleprompter = BootstrapFewShot(metric=metric, **config)
teleprompter.max_errors = 0
optimised_program = teleprompter.compile(AssessSubjectModule(), trainset=examples_for_training, valset=examples_for_testing)
# It doesnt use 'valset' for training (run in Jupyter loads automatically even if changed)

from dspy.evaluate import Evaluate
evaluate_program = Evaluate(metric=metric, devset=examples_for_testing, num_threads=1, display_progress=True, display_table=18)
evaluate_program(optimised_program)
# FIXME: argument 'bool' type is not iterable (for Evaluate); weird stuff.

100%|██████████| 8/8 [00:00<00:00, 111.38it/s]


Average Metric: 16 / 18  (88.9): 100%|██████████| 18/18 [00:00<00:00, 110.70it/s]


,smart_contract_description,example_boolean_assessment,reasoning,pred_boolean_assessment,metric
0,"Launching a successful fan engagement platform necessitates a robust business strategy. Initial funding might be sourced from venture capital, emphasizing the innovative use of blockchain...",False,produce the boolean_assessment. We should review the text for a description of a specific smart contract and its functionalities.,False,✔️ [True]
1,"In the design world, the concept of a 'Design Compiler' could function similarly to how smart contracts are compiled. This tool would automatically transform high-level...",False,produce the boolean_assessment. We need to review the text for a description of a specific smart contract and its functionalities.,False,✔️ [True]
2,"Educational platforms are exploring avenues similar to smart contracts for automating the issuance of certificates and managing course enrollments. By using predefined criteria, these platforms...",False,produce the boolean_assessment. We should review the text for a description of a specific smart contract and its functionalities.,True,False
3,"Smart contract programming involves writing code that is deployed on a blockchain. The most popular language for this is Solidity, used primarily for Ethereum. Smart...",False,"produce the boolean assessment. The provided text describes the process of smart contract programming, the language used (Solidity), and the compilation process. It does not...",False,✔️ [True]
4,Why did the smart contract apply to law school? Because it wanted to pass the bar â€” except it failed when it couldn't handle a...,False,"produce the boolean assessment. We should review the text for a description of a specific smart contract and its functionalities. Based on the description provided,...",False,✔️ [True]
5,"For the Fan Engagement Platform, design a ERC721 Smart Contract to distribute rewards based on activity and token holdings at the end of each month....",True,"produce the boolean_assessment. We should review the text for a description of a specific smart contract and its functionalities. Based on the description provided, it...",True,✔️ [True]
6,Strategic implementation of digital contracts in the healthcare sector could revolutionize patient engagement and data management. By setting predefined terms for data access and patient...,False,produce the boolean_assessment. We should review the text for a description of a specific smart contract and its functionalities.,True,False
7,Imagine a digital marketing campaign that functions like a smart contract. Campaign elements are executed automatically based on audience engagement metrics. This could include the...,False,produce the boolean_assessment. We should review the text for a description of a specific smart contract and its functionalities.,False,✔️ [True]
8,"The Digital Canvas initiative leverages the concept of programmable interactions, much like a smart contract, but for artists and galleries. The platform allows digital creators...",False,produce the boolean_assessment. We should review the text for a description of a specific smart contract and its functionalities.,False,✔️ [True]
9,"For the Fan Engagement Platform, design a ERC721 Smart Contract that rewards fans for referring friends, with bonuses based on the number of successful referrals....",True,produce the boolean assessment. The text provides a specific description of a smart contract for rewarding fans for referring friends and outlines the bonuses and...,True,✔️ [True]


88.89

In [65]:
from dspy.evaluate import Evaluate
evaluate_program = Evaluate(metric=metric, devset=examples_for_testing, num_threads=1, display_progress=True, display_table=None)
evaluate_program(optimised_program)
# FIXME: argument 'bool' type is not iterable (for Evaluate); weird stuff.

  0%|          | 0/18 [00:00<?, ?it/s]Defined function 'metric' called on
    Example({'smart_contract_description': 'Code binds agreement, Blocks chain truth in the ledger, Trust is automatic. Transactions secure, Ledger logs every action, Immutable trace. Contracts self-execute, Mistakes are past, trust in code, Blockchain does ensure.', 'boolean_assessment': False}) (input_keys={'smart_contract_description'})
Generated prediction is
    Prediction(
    reasoning='produce the boolean assessment. We will determine if the text provides a clear and precise description of how a specific smart contract should function, including the rules, actions, and features it carries out.',
    boolean_assessment=False
)
    >> Real-value/Predicted-value : False | False

Average Metric: 1 / 1  (100.0):   0%|          | 0/18 [00:00<?, ?it/s]Defined function 'metric' called on
    Example({'smart_contract_description': "Educational platforms are exploring avenues similar to smart contracts for automati

72.22

In [66]:
optimised_program.forward(".")

Prediction(
    reasoning='produce the boolean assessment. We can see that the text is incomplete and does not provide a specific description of how a smart contract should work.',
    boolean_assessment=False
)

In [67]:
import pydantic

# Pydantic model to define the output structure
class OutputRate(pydantic.BaseModel):
    parameter: int

class RateCompexity(dspy.Signature):
    """Rate the technical completeness of the smart contract description on a scale of 1 to 10"""
    smart_contract_description: str = dspy.InputField(desc="A description of a Smart Contract")
    complexity_score: OutputRate = dspy.OutputField(desc="A rating from 1 to 10")

class RateComplexityModule(dspy.Module):
    """A module to rate the technical completeness of a smart contract description"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedChainOfThought(RateCompexity)

    def forward(self, description: str) -> int:
        pred = self.generate_answer(smart_contract_description=description)
        return pred.complexity_score.parameter

# Example usage of the module
module = RateComplexityModule()
rating = module.forward(description)
print(rating)

# gpt3_turbo.inspect_history(n=1)


### COMPILATION ########################################################################

# must match "forward" function arguments
trainset = [
    dspy.Example(smart_contract_description=description, complexity_score=0).with_inputs("description"),
]

# first_data_point = trainset[0]
# print(first_data_point.inputs())
# print(first_data_point.labels())

from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
def validate_assessment(example, pred, trace=None):
    ### answer_EM = dspy.evaluate.answer_exact_match(smart_contract_description, complexity_score)
    answer_match = example.complexity_score == pred.complexity_score
    return answer_match

teleprompter = BootstrapFewShot(metric=validate_assessment) # or dspy.evaluate.answer_exact_amtch
compiled_module = teleprompter.compile(RateComplexityModule(), trainset=trainset)
# compiled_module.save(".")
# loaded_program = YOUR_PROGRAM_CLASS()
# loaded_program.load(path=YOUR_SAVE_PATH)

devset = [
    dspy.Example(smart_contract_description=description, complexity_score=5).with_inputs("description")
]

print('#')

from dspy.evaluate import Evaluate

# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Launch evaluation.
evaluator(compiled_module, metric=validate_assessment)

NameError: name 'description' is not defined

In [57]:
import pydantic
from typing import List
description = A1

class RequirementsList(pydantic.BaseModel):
    parameter: List[str]

class ExtractRequirements(dspy.Signature):
    """Extract requirements from a Smart Contract Description"""
    smart_contract_description: str = dspy.InputField(desc="A description of a Smart Contract")
    requirements_list: RequirementsList = dspy.OutputField(desc="A list object with extracted requirements")

class ExtractRequirementsModule(dspy.Module):
    """A module to extract requirements from a Smart Contract Description"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedPredictor(ExtractRequirements)

    def forward(self, description: str) -> RequirementsList:
        pred = self.generate_answer(smart_contract_description=description)
        return pred.requirements_list.parameter

module = ExtractRequirementsModule()
list_of_requirements = module.forward(description)
print(list_of_requirements)

# gpt3_turbo.inspect_history(n=1)

['buyer request refund', 'compensation based on current state of sale (in progress or cancelled)', 'seller authorization for refund initiation', 'refund amount calculation based on sale state', 'no refund if sale is not in progress']


In [42]:
import pydantic
from typing import List

class Requirement(pydantic.BaseModel):
    Name: str
    Scope: str
    Input: List[str]
    Constraints: str
    Output: str
    PrimaryScenario: str
    AlternativeScenario: str

class generate_attributes(dspy.Signature):
    """Generate attributes for the given attribute description."""
    smart_contract_description: str = dspy.InputField(desc="Context of the requirement")
    requirement_description: str = dspy.InputField()
    structured_requirement: Requirement = dspy.OutputField(desc="Structured list of requirement attributes")

class GenerateAttributesModule(dspy.Module):
    """A module to process multiple requirement descriptions into structured JSON."""

    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.functional.TypedPredictor(generate_attributes)
    
    def forward(self, description: str, requirement: str) -> Requirement:
        pred = self.generate_answer(
            smart_contract_description=description,
            requirement_description=requirement
            )
        return pred.structured_requirement

req_module = GenerateAttributesModule()
# List of requirements has multiple strings inside.
for requirement in list_of_requirements:
    attributes = req_module.forward(description, requirement)
    print(attributes.dict())

# gpt3_turbo.inspect_history(n=1)

{'Name': 'Buyer Refund Request', 'Scope': 'Smart contract for refund process', 'Input': ['Buyer request', 'Sale status'], 'Constraints': 'Only seller can initiate refund process', 'Output': 'Fair compensation for buyer', 'PrimaryScenario': 'If sale in progress, refund of twice the initial value', 'AlternativeScenario': 'No refund if sale not in progress'}
{'Name': 'Compensation based on current state of sale', 'Scope': 'Smart contract refund process', 'Input': ['Buyer request for refund', 'Current state of the sale (in progress or cancelled)'], 'Constraints': 'Only seller authorized to initiate refund process', 'Output': 'Fair compensation for buyer if sale is cancelled', 'PrimaryScenario': 'If sale is in progress and seller decides not to proceed, buyer receives a refund of twice the value paid initially', 'AlternativeScenario': 'No refund issued if sale is not in progress'}
{'Name': 'Seller Authorization for Refund Initiation', 'Scope': 'Smart Contract for Buyer Refunds', 'Input': ['